In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir=(f"/content/drive/MyDrive/Major/flowers/flowers")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
def data_augment(image):
  p_spatial =tf.random.uniform([],0,1.0, dtype=tf.float32)
  p_rotate =tf.random.uniform([],0,1.0, dtype=tf.float32)
  p_pixel_1 =tf.random.uniform([],0,1.0, dtype=tf.float32)
  p_pixel_2 =tf.random.uniform([],0,1.0, dtype=tf.float32)
  p_pixel_3 =tf.random.uniform([],0,1.0, dtype=tf.float32)

  #Flips
  image=tf.image.random_flip_left_right(image)
  image=tf.image.random_flip_up_down(image)

  if p_spatial >0.75:
    image=tf.image.transpose(image)

  #Rotates
  if p_rotate> 0.75:
    image=tf.image.rot90(image,k=3) #rotate 270
  elif p_rotate > 0.5:
    image=tf.image.rot90(image,k=2) #rotate 180
  elif p_rotate > 0.25:
    image=tf.image.rot90(image,k=1) #rotate 90

  #Pixel_Level tramsform:
  if p_pixel_1>= 0.4:
     image=tf.image.random_saturation(image,lower=0.7,upper=1.3)
  if p_pixel_2>= 0.4:
     image=tf.image.random_contrast(image,lower=0.8,upper=1.2)
  if p_pixel_3>= 0.4:
     image=tf.image.random_brightness(image, max_delta=0.1)

  return image

In [ ]:
#import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)
train_datagen=ImageDataGenerator(rescale=1/255.,
                                 validation_split= 0.2,
                                 preprocessing_function =data_augment
                                 )
# test_datagen =ImageDataGenerator(rescale=1/255.)

train_data= train_datagen.flow_from_directory(train_dir,target_size=(224,224), subset="training",batch_size=64)
test_data= train_datagen.flow_from_directory(train_dir,target_size=(224,224), subset="validation",batch_size=64,shuffle=False)

In [ ]:
from keras.layers import Flatten,Dense, Conv2D, AveragePooling2D, Dropout, MaxPool2D, Rescaling ,BatchNormalization, Activation,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
#copy the  predefind model and use imagenet weight
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
 include_top=False)
#use global average layer
avg =tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# use hidden layer of 24 neuron
layer = tf.keras.layers.Dense(20, activation="selu")(avg)
layer = tf.keras.layers.Dropout(0.5)(layer)
output = tf.keras.layers.Dense(5, activation="softmax")(layer)
model_imagnet = tf.keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
#first freez the weight of predefind model
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
#use Stochastic gradient optimizers
opt = tf.keras.optimizers.SGD(learning_rate=0.0005,momentum=0.9,decay = 0.01)

In [ ]:
model_imagnet.compile(optimizer=opt, loss = "categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history_imgnet = model_imagnet.fit(train_data,
                    epochs=10,
                    validation_data=test_data)

In [ ]:
earlystopping=tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                               min_delta =1e-4,
                                               patience=8,
                                               mode='max',
                                               restore_best_weights =True,
                                               verbose=1)
callbacks=[earlystopping]

In [ ]:
for layer in base_model.layers:
  layer.trainable = True

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07)

In [ ]:
model_imagnet.compile(optimizer=opt, loss = "categorical_crossentropy", metrics=['accuracy'])

In [ ]:
history_imgnet1 = model_imagnet.fit(train_data,
                    epochs=50,
                    validation_data=test_data,
                    callbacks=callbacks
                    )

In [ ]:
model_imagnet.save("/content/drive/MyDrive/Major/flowers/model_save")

In [ ]:
tf.keras.models.save_model(model_imagnet,"/content/drive/MyDrive/Major/flowers/model_imagenet")

In [ ]:
import matplotlib.pyplot as plt
l = len(history_imgnet.history["loss"])+len(history_imgnet1.history["loss"])
t_acc = history_imgnet.history["accuracy"]+history_imgnet1.history["accuracy"]
v_acc = history_imgnet.history["val_accuracy"]+history_imgnet1.history["val_accuracy"]
plt.plot([x for x in range(1,l+1)],t_acc,c="red")
plt.plot([x for x in range(1,l+1)],v_acc,c="green")

In [ ]:
t_loss = history_imgnet.history["loss"]+history_imgnet1.history["loss"]
v_loss = history_imgnet.history["val_loss"]+history_imgnet1.history["val_loss"]
plt.plot([x for x in range(1,l+1)],t_loss,c="red")
plt.plot([x for x in range(1,l+1)],v_loss,c="green")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predicted_classes=np.argmax(model_imagnet.predict(test_data, steps=test_data.n // test_data.batch_size+1), axis=-1)
true_classes=test_data.classes
#class_labels=list(test_data.class_indices.keys())

confusionmatrix=tf.math.confusion_matrix(true_classes,predicted_classes)
plt.figure(figsize=(16,16))
sns.heatmap(confusionmatrix, cmap='Blues', annot =True ,cbar=True)

print(classification_report(true_classes,predicted_classes))